In [1]:
import pandas as ps
from urllib.request import urlopen
import requests
url = "https://raw.githubusercontent.com/micybusy/sketch/main/charts.csv"
with urlopen(url) as f:
    content = ps.read_csv(url, encoding = "utf-8")
content["rankimportance"]=101-content["rank"]
content

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,rankimportance
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,100
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,99
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,98
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,97
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,96
...,...,...,...,...,...,...,...,...
330082,1958-08-04,96,Over And Over,Thurston Harris,NaN,96,1,5
330083,1958-08-04,97,I Believe In You,Robert & Johnny,NaN,97,1,4
330084,1958-08-04,98,Little Serenade,The Ames Brothers,NaN,98,1,3
330085,1958-08-04,99,I'll Get By (As Long As I Have You),Billy Williams,NaN,99,1,2
